In [1]:
import pandas as pd

from fluxo.dados import Dados
from fluxo.carga_dados import CargaDados
from fluxo.limpeza_dados import LimpezaDados
from fluxo.preprocessamento import Preprocessamento
from fluxo.extracao_caracteristicas import ExtracaoCaracteristicas
from fluxo.treinamento import Treinamento
from fluxo.classificador import Classificador

from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
pd.set_option('max_colwidth', 180)

## Carga de Dados

In [3]:
dados = Dados()

In [4]:
cg = CargaDados()
cg.executar(dados)

INFO:root:Carregando dados de segmentos...
INFO:root:206488 registros carregados.


## Limpeza de Dados

In [5]:
lp = LimpezaDados(dados)
lp.executar(dados)

INFO:root:Excluindo segmentos dos atos que não são ADE...
INFO:root:91448 segmentos de atos não ADE excluídos.
INFO:root:Restaram 115040 segmentos de atos ADE.
INFO:root:Removendo segmentos não representativos...
INFO:root:6827 segmentos não representativos excluídos.
INFO:root:Restaram 108213 segmentos representativos.
INFO:root:Removendo segmentos nulos...
INFO:root:69 segmentos nulos excluídos.
INFO:root:Restaram 108144 segmentos não nulos.
INFO:root:Removendo tags HTML...
INFO:root:Removendo caracteres de escape HTML...
INFO:root:Reclassificando segmentos não identificados...
INFO:root:1418 segmentos reclassificados como artigos.
INFO:root:Limpeza de dados concluída.


## Preprocessamento

In [6]:
pp = Preprocessamento()
pp.executar(dados)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/91872936504/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
INFO:root:Convertendo caracteres para minúsculo...
INFO:root:Removendo pontuação...
INFO:root:Realizando tokenização...
INFO:root:Removendo stopwords...
INFO:root:Removendo tokens menores que 2 caracteres...
INFO:root:Reconstruindo texto a partir dos tokens...
INFO:root:Preprocessamento concluído.


## Extração de Características

In [7]:
ec = ExtracaoCaracteristicas(dados)
ec.executar(dados)

INFO:root:Executando vetorização TF-IDF...
INFO:root:Executando vetorização TF-IDF para Cross-Validation...
INFO:root:Extração de características concluída.


## Treinamento e Teste

In [8]:
t = Treinamento()

In [9]:
# Linear SVC (SVM)
estimador = OneVsOneClassifier(LinearSVC())
t.adicionar_modelo(Classificador('LinearSVC-1x1', estimador))
t.treinar_modelos(dados)

estimador = OneVsRestClassifier(LinearSVC())
t.adicionar_modelo(Classificador('LinearSVC-1xR', estimador))
t.treinar_modelos(dados)

INFO:root:Treinando modelos...
INFO:root:Treinando modelos...


In [10]:
# Multinomial Naive Bayes 
estimador = OneVsOneClassifier(MultinomialNB())
t.adicionar_modelo(Classificador('MultinomialNB-1x1', estimador))
t.treinar_modelos(dados)

estimador = OneVsRestClassifier(MultinomialNB())
t.adicionar_modelo(Classificador('MultinomialNB-1xR', estimador))
t.treinar_modelos(dados)

INFO:root:Treinando modelos...
INFO:root:Treinando modelos...


## Treinamento e Teste com Cross-Validation

In [11]:
# Linear SVC (SVM)
estimador = OneVsOneClassifier(LinearSVC())
t.adicionar_modelo(Classificador('LinearSVC-1x1-CV', estimador))
t.treinar_modelos(dados, 5)

estimador = OneVsRestClassifier(LinearSVC())
t.adicionar_modelo(Classificador('LinearSVC-1xR-CV', estimador))
t.treinar_modelos(dados, 5)

INFO:root:Treinando modelos com cross-validation...
INFO:root:Treinando modelos com cross-validation...


In [12]:
# Multinomial Naive Bayes 
estimador = OneVsOneClassifier(MultinomialNB())
t.adicionar_modelo(Classificador('MultinomialNB-1x1-CV', estimador))
t.treinar_modelos(dados, 5)

estimador = OneVsRestClassifier(MultinomialNB())
t.adicionar_modelo(Classificador('MultinomialNB-1xR-CV', estimador))
t.treinar_modelos(dados, 5)

INFO:root:Treinando modelos com cross-validation...
INFO:root:Treinando modelos com cross-validation...


In [13]:
resultado = pd.DataFrame(dados.metricas)
resultado.transpose()

,acuracia,precisao,revocacao,f1
LinearSVC-1x1,0.9766,0.9679,0.9663,0.9668
LinearSVC-1xR,0.9750,0.9643,0.9667,0.9651
MultinomialNB-1x1,0.9249,0.9110,0.9170,0.9127
MultinomialNB-1xR,0.9265,0.9164,0.9185,0.9163
LinearSVC-1x1-CV,0.9766,0.9679,0.9663,0.9668
LinearSVC-1xR-CV,0.9750,0.9643,0.9667,0.9651
MultinomialNB-1x1-CV,0.9249,0.9110,0.9170,0.9127
MultinomialNB-1xR-CV,0.9265,0.9164,0.9185,0.9163
